# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

In [3]:
print(twits['data'][:5])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}]


### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [4]:
"""print out the number of twits"""

# DONE Implement 
print(len(twits['data']))

1548010


### Split Message Body and Sentiment Score

In [5]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

In [6]:
print(messages[:5])

['$FITB great buy at 26.00...ill wait', '@StockTwits $MSFT', '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', '$AMD reveal yourself!']


In [7]:
print(sentiments[:5])

[4, 3, 4, 3, 2]


## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [8]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #DONE: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    text = re.sub(url_pattern, ' ', text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    ticker_pattern = r'\$[a-z]+'
    text = re.sub(ticker_pattern, ' ', text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    username_pattern = r'\@[a-z0-9]+'
    text = re.sub(username_pattern, ' ', text)

    # Replace everything not a letter with a space
    notletter_pattern = r'[^a-z]'
    text = re.sub(notletter_pattern, ' ', text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split()

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(wnl.lemmatize(w,'n'),'v') for w in tokens if len(w) > 1]

    
    assert type(tokens) == list, 'Tokens should be list'
    return tokens

[nltk_data] Downloading package wordnet to /home/student/nltk_data...


In [9]:
preprocess('RT @google1 Our 2024 annual look at a the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG')

['rt',
 'our',
 'annual',
 'look',
 'at',
 'the',
 'year',
 'in',
 'google',
 'blogging',
 'and',
 'beyond']

>Note: You must ensure that after preprocessing the text should NOT include:
- Numbers
- URLs
- Single character tokens
- Ticker symbols (these should be removed even if they don't appear at the beginning)

### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [10]:
# DONE Implement

tokenized = [preprocess(message) for message in messages]

In [11]:
tokenized[:5]

[['great', 'buy', 'at', 'ill', 'wait'],
 [],
 ['staanalystalert',
  'for',
  'jefferies',
  'maintain',
  'with',
  'rat',
  'of',
  'hold',
  'set',
  'target',
  'price',
  'at',
  'usd',
  'our',
  'own',
  'verdict',
  'be',
  'buy'],
 ['hear',
  'there',
  'guy',
  'who',
  'know',
  'someone',
  'who',
  'think',
  'somebody',
  'know',
  'something',
  'on',
  'stocktwits'],
 ['reveal', 'yourself']]

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [12]:
from collections import Counter
"""
Create a vocabulary by using Bag of words
"""

# DONE: Implement 
flattened_tokens = [word for twit in tokenized for word in twit]
word_counts = Counter(flattened_tokens)

In [13]:
for word, count in word_counts.items():
    print(f"{word}: {count}")

great: 17574
buy: 119297
at: 138469
ill: 883
wait: 19014
staanalystalert: 4072
for: 273556
jefferies: 1056
maintain: 1849
with: 68867
rat: 16027
of: 211382
hold: 39784
set: 9732
target: 16981
price: 40063
usd: 2959
our: 12209
own: 10961
verdict: 3736
be: 526989
hear: 3829
there: 25945
guy: 13438
who: 17885
know: 22692
someone: 5192
think: 34391
somebody: 1100
something: 6010
on: 241679
stocktwits: 2034
reveal: 489
yourself: 1207
why: 21708
the: 398919
drop: 23039
warren: 1015
buffet: 1304
take: 30186
out: 56718
his: 6261
position: 17479
bear: 33765
have: 64958
reason: 7866
to: 379537
pay: 11379
more: 55375
attention: 3531
ok: 4567
good: 44208
we: 64758
re: 13194
not: 68312
in: 205343
over: 32685
weekend: 3561
lol: 30941
daily: 14036
chart: 16195
need: 23065
get: 75486
back: 43516
above: 17856
per: 16565
week: 44057
after: 34400
spike: 2539
if: 59001
no: 40435
news: 19742
month: 16914
bo: 610
then: 20975
bingo: 181
what: 79153
odds: 392
strong: 17202
short: 93857
ratio: 7664
and: 208527

shitake: 1
newz: 3
monnn: 3
aaaaay: 2
itttttt: 6
fusion: 26
daaaaaaamn: 4
symmetry: 16
riseeeee: 3
faaaaaaaack: 1
faaaack: 1
senseless: 31
laaaaawdddeee: 1
geeebus: 1
wowowowo: 1
gracias: 17
izzz: 1
boooyahhh: 1
greeeeeeennn: 1
yayyyyyy: 3
iiiinnncrreedddiibblleee: 1
gaaawd: 1
zillllllllaaaah: 1
wold: 13
craziest: 37
smddh: 1
kari: 4
toady: 7
holyyyy: 2
insititutions: 2
maroon: 3
daq: 23
puuuuuusshh: 1
tantric: 2
damnnnnnn: 10
paralysis: 2
netflixed: 3
roids: 3
canonical: 2
broooooooo: 2
moooo: 15
wazzupppp: 1
ttetectech: 1
hmis: 1
unabashedly: 2
alotta: 3
unquenchable: 1
haaappy: 1
boon: 25
incroyable: 1
ojhgsdohgfoisjoighjshgihsepihgpiwegpijhsegjwejgopisdhgopihse: 1
gtw: 1
llike: 3
daaaamnnnn: 1
ridicilous: 6
ooooooohhhhhhh: 1
sharper: 10
ahhhhhhhh: 7
fuktun: 1
waa: 7
loke: 3
fuggetaboutit: 2
parody: 5
scurrred: 3
reconsolidating: 1
roundprints: 1
lasvegasshooting: 1
highprobchannel: 1
loaaaad: 1
hafta: 5
jimjam: 1
gooooooooooooookooopok: 1
redsox: 4
positon: 10
overcook: 6
gaaaaaaa:

upstate: 5
stmnts: 1
chanl: 1
liter: 1
irreplaceable: 1
seagal: 5
apptd: 1
ferazzi: 11
exovest: 1
papapapapapapapa: 1
greeeeeeeen: 6
ruhtroh: 1
iris: 7
overhump: 1
speaka: 1
tredged: 1
puleeze: 1
lookingfor: 3
robbin: 4
yinz: 4
moajority: 1
annoymous: 1
turnarounded: 1
dunnnnnnnn: 1
dumpfest: 3
eyelea: 2
comuunication: 1
flusshhhhhh: 1
timewarped: 1
formulary: 6
zepatie: 1
zepati: 2
lvcnyi: 3
threadrippered: 3
howareyoupositioned: 10
boooooooya: 1
faceprint: 1
letitride: 2
stockideas: 2
yieldleaders: 2
moi: 2
boooooooooooooski: 1
dooooooooooooomski: 1
intelripper: 3
pcr: 1
ohboy: 1
pif: 11
buicks: 6
feebleness: 1
wamts: 1
targetung: 1
crazt: 1
extrapolation: 3
jerrryyy: 1
willow: 2
marilyn: 4
roceeds: 1
crazzzy: 1
tpus: 1
inevitability: 5
whenver: 3
ahhb: 1
breakneck: 4
asmr: 2
facetious: 2
woolworth: 5
newbury: 2
mediate: 2
stubles: 1
deop: 3
computerize: 6
seykota: 1
weeklong: 4
oneteam: 1
ornot: 1
heartstone: 2
gemphire: 1
selloffis: 1
provate: 1
poltical: 1
marriot: 4
tarcking: 1
j

ttid: 4
svod: 2
ejaculation: 2
apos: 137
boooooooooommmmmmmm: 1
atmfh: 1
tropic: 3
yooooooooooooooo: 1
succeful: 1
xiomai: 1
thf: 1
ecigs: 1
gogogogoggoo: 1
fangled: 2
gaing: 1
masyer: 1
rocketttttttt: 1
eeeeeeeeeeeeeeeeeeeee: 1
eeeeeeeeeeeeeeeeee: 1
forstall: 1
portfilio: 1
maxgate: 1
becn: 1
peaceout: 1
ozone: 2
eeeeeeeeeee: 1
inmpulse: 1
surmize: 2
hahahahshahahahahaha: 1
shortards: 1
hahahahahahahahahahahahahahahahahahahahahahahahahah: 1
amdeeze: 1
somegood: 1
cinematography: 1
arpril: 1
hoodrich: 1
teleporter: 1
incumming: 1
launchaaaa: 1
theclicker: 1
euphoriaeuphoriaeuphoria: 1
misspeak: 1
ipants: 1
ishirt: 1
theese: 1
cccaaannnt: 1
medman: 2
zmomentum: 1
uswa: 1
halle: 1
luja: 1
gagillion: 2
sellspreads: 1
morreee: 1
nlow: 1
throug: 1
itsstillcrashing: 1
bankruptsees: 1
kicka: 1
comeee: 1
unchain: 2
fancier: 1
acagyxfl: 1
boohahahaha: 1
russkis: 1
bayblade: 1
saphire: 1
hdp: 1
mildew: 1
perimeter: 1
baaaaaa: 1
aposs: 796
everytine: 1
yaawwwnnn: 1
narrowminded: 1
hypeeeee: 1
pan

injan: 1
fuxnin: 1
gonnn: 1
crssh: 1
ehrlich: 3
screwwd: 1
isjust: 1
technicaldominance: 1
faanngsrdead: 1
fuhcin: 1
wtse: 1
robyn: 1
nutritionally: 1
caloric: 1
shoted: 1
pucjer: 1
pricetech: 1
signficant: 1
jeebezus: 1
beastt: 1
thougjt: 1
burnnning: 1
beezos: 2
zeglis: 2
bleeing: 1
revlmd: 1
opporunities: 1
khrist: 1
obergruppenf: 1
xxi: 1
lfmaoo: 1
faiiilll: 1
holyyyyyyyy: 1
overinvest: 1
nissin: 1
gallaga: 1
tendancy: 1
greatttttttttt: 1
ghanta: 1
tofu: 1
sofritto: 1
hiper: 1
ummmummmm: 1
ttttttank: 1
fukboy: 1
broadcating: 1
umming: 1
sandro: 1
pathatik: 1
bezzossssssss: 1
guyy: 1
yed: 1
rigjt: 2
riiight: 1
notate: 1
rko: 1
fute: 1
gahd: 1
cloase: 1
tamharaw: 1
chipo: 1
institutionalise: 1
concall: 1
rebounces: 1
whilepayouts: 1
geveryone: 1
havy: 1
whoopsey: 1
yayayyayayay: 2
anwer: 1
narrorw: 1
thiswar: 1
wouldntquit: 1
streeters: 1
exect: 1
voo: 2
yayayayayayyay: 1
proprob: 1
texwreck: 1
gwdc: 2
nchange: 1
nanowire: 2
yayayayyay: 1
amatuerhour: 1
untangle: 2
beause: 1
moor: 1


In [14]:
print(word_counts['and'])

208527


### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the **number of messages** calculated in the code block above (i.e. `len(messages))`. Then remove words that only appear in some small fraction of the messages.

>Note: There is not an exact number for low and high-frequency cut-offs, however there is a correct optimal range.
You should ideally set up low-frequency cut-off from 0.0000002 to 0.000007 (inclusive) and high-frequency from 5 to 20 (inclusive). If the number is too big, we lose lots of important words that we can use in our data.

In [16]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# DONE Implement 

# Dictionary that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
num_messages = len(tokenized)
freqs = {word: count / num_messages for word, count in word_counts.items()}

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 0.000002

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 0.1

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [word for word in freqs if freqs[word] > high_cutoff]


filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)
print(len(word_counts))
print(len(filtered_words) )

['for', 'of', 'be', 'on', 'the', 'to', 'in', 'and', 'this', 'it']
92105
27381


### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [17]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

# DONE Implement

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {word: idx for idx, word in enumerate(filtered_words)}

# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {idx: word for word, idx in vocab.items()}

# tokenized with the words not in `filtered_words` removed.
filtered = [[word for word in twit if word in vocab] for twit in tokenized]

assert set(vocab.keys()) == set(id2vocab.values()), 'Check vocab and id2vocab dictionaries'

### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [18]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [19]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.19423672640468145

Finally let's convert our tokens into integer ids which we can pass to the network.

In [20]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

In [21]:
token_ids[:5]

[[0, 1, 2, 3, 4],
 [5, 6, 7, 8, 9, 10, 11, 12, 13, 2, 14, 15, 16, 17, 1],
 [18, 19, 20, 21, 22, 23, 21, 24, 25, 22, 26, 27],
 [30, 31, 32, 33, 34, 35, 36, 37],
 [38, 39, 40, 41, 42, 43]]

In [22]:
sentiments[:5]

[4, 4, 3, 3, 0]

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [23]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # DONE Implement

        # Setup embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)
        
        # Setup additional layers
        # LSTM layer
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers, 
                            dropout=dropout, batch_first=True)
        
        # FC layer
        self.fc = nn.Linear(lstm_size, output_size)
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout)
        


    def init_hidden(self, batch_size, device):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # DONE Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().to(device),
                  weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().to(device))
        
        return hidden
    


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        # DONE Implement 
        nn_input = nn_input.to(next(self.parameters()).device)
        
        # Pass input through the embedding layer
        embeds = self.embedding(nn_input)
        
        # Pass the embeddings and hidden state into the LSTM layer
        lstm_out, hidden_state = self.lstm(embeds, hidden_state)
        
        # Pass the LSTM output to the fully connected layer
        #lstm_out = lstm_out.contiguous().view(-1, self.lstm_size)
        lstm_out = lstm_out[-1, :, :]  # shape: (batch_size, lstm_size)
        
        # Dropout layer
        out = self.dropout(lstm_out)
        
        # Fully connected layer
        out = self.fc(out)
        
        # Apply softmax to the output
        logps = F.log_softmax(out, dim=1)
        
        return logps, hidden_state

### View Model

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.to(device)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(input.size(0), device)

logps, _ = model.forward(input, hidden)
print(logps)

tensor([[-1.9146, -1.8861, -1.1249, -1.5224, -1.8447],
        [-1.9025, -1.8895, -1.1193, -1.5301, -1.8537],
        [-1.8819, -1.9177, -1.0898, -1.5520, -1.8798],
        [-1.8722, -1.9412, -1.0742, -1.5629, -1.8866]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward0>)


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [25]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [26]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# DONE Implement 

split_frac = 0.8

split_idx = int(len(token_ids) * split_frac)

train_features = token_ids[:split_idx]
valid_features = token_ids[split_idx:]
train_labels = sentiments[:split_idx]
valid_labels = sentiments[split_idx:]

In [27]:
# Verify DataLoader Output
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))

# Move text_batch and labels to device
text_batch, labels = text_batch.to(device), labels.to(device)

model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.).to(device)
hidden = model.init_hidden(text_batch.size(0), device)
logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(27382, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [29]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 5
batch_size = 512
learning_rate = 0.001
sequence_length = 20

print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    #hidden = model.init_hidden(batch_size, device)
    steps = 0
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=sequence_length, shuffle=True):
        steps += 1
        #hidden = tuple([each.data.to(device) for each in hidden])
        hidden = model.init_hidden(text_batch.size(0), device)
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        
        # DONE Implement: Train Model
        
        # Zero gradients
        optimizer.zero_grad()
        #model.zero_grad()
        
        # Forward pass
        output, hidden = model(text_batch, hidden)
        
        # Calculate the loss and backpropagate
        loss = criterion(output, labels)
        loss.backward()
        
        # Clip gradients to avoid exploding gradients
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
        
        # Update weights
        optimizer.step()
        
        if steps % print_every == 0:
            model.eval()
            
            # DONE Implement: Print metrics
            # Validation
            val_losses = []
            #val_hidden = model.init_hidden(batch_size, device)
            
            for val_inputs, val_labels in dataloader(valid_features, valid_labels, batch_size=batch_size, sequence_length=sequence_length, shuffle=False):
                val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
                
                val_hidden = model.init_hidden(val_inputs.size(0), device)
                #val_hidden = tuple([each.data for each in val_hidden])
                
                with torch.no_grad():
                    val_output, val_hidden = model(val_inputs, val_hidden)
                
                val_loss = criterion(val_output, val_labels)
                val_losses.append(val_loss.item())
                
            
            print(f"Epoch: {epoch+1}/{epochs} | Step: {steps} | "
                  f"Train Loss: {loss:.4f} | "
                  f"Validation Loss: {val_loss:.4f} | ")
                        
            model.train()

Starting epoch 1
Epoch: 1/5 | Step: 100 | Train Loss: 1.3999 | Validation Loss: 1.3681 | 
Epoch: 1/5 | Step: 200 | Train Loss: 1.3692 | Validation Loss: 1.3846 | 
Epoch: 1/5 | Step: 300 | Train Loss: 1.3892 | Validation Loss: 1.3718 | 
Epoch: 1/5 | Step: 400 | Train Loss: 1.3736 | Validation Loss: 1.3495 | 
Epoch: 1/5 | Step: 500 | Train Loss: 1.3294 | Validation Loss: 1.3747 | 
Epoch: 1/5 | Step: 600 | Train Loss: 1.4051 | Validation Loss: 1.3578 | 
Epoch: 1/5 | Step: 700 | Train Loss: 1.3796 | Validation Loss: 1.3744 | 
Epoch: 1/5 | Step: 800 | Train Loss: 1.3902 | Validation Loss: 1.3495 | 
Epoch: 1/5 | Step: 900 | Train Loss: 1.3438 | Validation Loss: 1.3542 | 
Epoch: 1/5 | Step: 1000 | Train Loss: 1.3449 | Validation Loss: 1.3616 | 
Epoch: 1/5 | Step: 1100 | Train Loss: 1.3827 | Validation Loss: 1.3439 | 
Epoch: 1/5 | Step: 1200 | Train Loss: 1.3358 | Validation Loss: 1.3357 | 
Epoch: 1/5 | Step: 1300 | Train Loss: 1.3659 | Validation Loss: 1.3532 | 
Epoch: 1/5 | Step: 1400 | Trai

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [30]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # Done Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [word for word in tokens if word in vocab]
    # Convert words to ids
    tokens = [vocab[word] for word in tokens]
        
    # Adding a batch dimension
    text_input = torch.tensor(tokens).unsqueeze(0).to(next(model.parameters()).device)
    # Get the NN output
    hidden = model.init_hidden(1, next(model.parameters()).device)
    model.eval()
    with torch.no_grad():
        logps, _ = model(text_input, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    pred = pred.mean(dim=0)
    
    return pred

In [31]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

tensor([0.0618, 0.1215, 0.1965, 0.4217, 0.1985])

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
The prediction is 3 (positive review) with 42.17% certainty. The uncertainty is 57.83% (1 - 0.4217).

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [32]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [33]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [34]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [35]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'symbol': '$AAPL',
 'score': tensor([0.1057, 0.1202, 0.2646, 0.2772, 0.2324]),
 'timestamp': '2018-11-01T00:00:18Z'}

In [36]:
next(score_stream)

{'symbol': '$AMZN',
 'score': tensor([0.1481, 0.1270, 0.2486, 0.2840, 0.1923]),
 'timestamp': '2018-11-01T00:00:42Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.